In [2]:
import os
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# --- texture classes you asked for ---
TEXTURE_CLASSES = [
    "Loam",
    "Sandy Clay Loam",
    "Sandy Loam",
    "Sand",
    "Clay",
    "Sandy Clay",
]

# --- pick a consistent style (color + hatch) per class ---
# Feel free to change these; the important part is: one style per class.
TEXTURE_STYLE = {
    "Loam":            dict(facecolor="tan",        hatch=".."),
    "Sandy Clay Loam": dict(facecolor="peru",       hatch="xx"),
    "Sandy Loam":      dict(facecolor="wheat",      hatch="//"),
    "Sand":            dict(facecolor="khaki",      hatch="oo"),
    "Clay":            dict(facecolor="sienna",     hatch="\\\\"),
    "Sandy Clay":      dict(facecolor="chocolate",  hatch="++"),
}

def _slugify(name: str) -> str:
    name = name.strip().lower()
    name = re.sub(r"\s+", "_", name)
    name = re.sub(r"[^a-z0-9_]+", "", name)
    return name

def make_texture_swatch_pngs(
    out_dir="texture_png",
    width_px=900,
    height_px=260,
    dpi=200,
    edgecolor="black",
    linewidth=2.0,
    fontsize=26,
):
    os.makedirs(out_dir, exist_ok=True)

    # compute figure size in inches from pixel target
    fig_w = width_px / dpi
    fig_h = height_px / dpi

    for tex in TEXTURE_CLASSES:
        if tex not in TEXTURE_STYLE:
            raise KeyError(f"Missing style for texture class: {tex}")

        style = TEXTURE_STYLE[tex]

        fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=dpi)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis("off")

        # main swatch rectangle
        rect = Rectangle(
            (0.02, 0.08), 0.96, 0.84,
            facecolor=style["facecolor"],
            edgecolor=edgecolor,
            linewidth=linewidth,
            hatch=style.get("hatch", None),
        )
        ax.add_patch(rect)

        # centered label
        ax.text(
            0.5, 0.5, tex,
            ha="center", va="center",
            fontsize=fontsize,
            color="black",
            weight="bold",
        )

        fname = f"{_slugify(tex)}.png"
        out_path = os.path.join(out_dir, fname)
        fig.savefig(out_path, transparent=True, bbox_inches="tight", pad_inches=0.02)
        plt.close(fig)

        print(f"Saved: {out_path}")

make_texture_swatch_pngs()


Saved: texture_png/loam.png
Saved: texture_png/sandy_clay_loam.png
Saved: texture_png/sandy_loam.png
Saved: texture_png/sand.png
Saved: texture_png/clay.png
Saved: texture_png/sandy_clay.png


In [3]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch

def plot_texture_profile(horizons, ax=None, title="USDA Texture by Depth"):
    """
    horizons: list of dicts like:
      [{"top_cm": 0, "bottom_cm": 15, "texture": "Sandy Loam"}, ...]
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(3, 7), dpi=200)
    else:
        fig = ax.figure

    # single "column" width in x
    x0, w = 0.0, 1.0

    for h in horizons:
        top = float(h["top_cm"])
        bot = float(h["bottom_cm"])
        tex = h["texture"].strip()

        if tex not in TEXTURE_STYLE:
            raise KeyError(f"Texture '{tex}' not in TEXTURE_STYLE. Add it or fix spelling.")

        style = TEXTURE_STYLE[tex]
        height = bot - top

        ax.add_patch(Rectangle(
            (x0, top), w, height,
            facecolor=style["facecolor"],
            edgecolor="black",
            linewidth=1.2,
            hatch=style.get("hatch", None)
        ))

        # optional label inside each layer
        ax.text(
            x0 + w/2, top + height/2, tex,
            ha="center", va="center",
            fontsize=9
        )

    ax.set_xlim(0, 1)
    ax.set_xticks([])
    ax.set_ylabel("Depth (cm)")
    ax.set_title(title)

    # Depth increases downward
    ax.invert_yaxis()

    # Legend (unique classes present)
    present = []
    for h in horizons:
        t = h["texture"].strip()
        if t not in present:
            present.append(t)

    handles = [
        Patch(facecolor=TEXTURE_STYLE[t]["facecolor"],
              edgecolor="black",
              hatch=TEXTURE_STYLE[t].get("hatch", None),
              label=t)
        for t in present
    ]
    ax.legend(handles=handles, loc="upper right", bbox_to_anchor=(1.65, 1.0))

    return fig, ax

# Example horizons (replace with your field data)
horizons = [
    {"top_cm": 0,  "bottom_cm": 10, "texture": "Sandy Loam"},
    {"top_cm": 10, "bottom_cm": 35, "texture": "Loam"},
    {"top_cm": 35, "bottom_cm": 60, "texture": "Sandy Clay Loam"},
    {"top_cm": 60, "bottom_cm": 90, "texture": "Clay"},
]

fig, ax = plot_texture_profile(horizons)
plt.show()
